In [ ]:
# std lib
import os
import pickle
import sys

# 3rd party lib
import h5py
import numpy as np
import glob
import os
import resampy

#加一个高通滤波
from scipy import signal

In [ ]:
#loading file
f = h5py.File("/p300/PATH_OF_RAW_DATA_LABELED_TEST.h5",'r')
f.keys()# <KeysViewHDF5 ['filenames', 'labels', 'signals']>


In [ ]:
a = f['labels'][31]
#sampling frequency 200Hz
#minimum seizure period is 15s
#find the start and end time stamp in signal[i]
np.max(np.where(a==1))-np.min(np.where(a==1))
np.size(np.where(a==1))


In [ ]:
threshold = [2*0.75/200,2*40/200] #2*0.75Hz/200Hz
b, a = signal.butter(6, threshold, 'bandpass')   #配置滤波器 4 表示滤波器的阶数
# X = signal.filtfilt(b, a, X)

In [ ]:
#find seizure EEG data
min_period = 60
frequency = 200
channel = 18
suit_list = list()#suitable_object_list
for i in range(500):
    signal = f['labels'][i]
    if np.size(np.where(signal==1)) != 0:
        time_period = np.max(np.where(signal==1))-np.min(np.where(signal==1))
        if time_period > min_period*frequency:
            suit_list.append([i,np.min(np.where(signal==1)),np.max(np.where(signal==1))])
# there are 51 objects which is satisfied conditions in first 200 

In [ ]:
#seizure data
# eeg_data_list = list()#store the data
data_list = list()
label_list = list()#zero is normal and one is seizure fragment
for tmp_list in suit_list:
    i = tmp_list[0]
    min_value = tmp_list[1]
    max_value = tmp_list[2]
    # solving the special data structure array in array
    # out_signal = signal.filtfilt(b, a, f['signals'][i])
    # out_signal = f['signals'][i]
    out = np.concatenate(f['signals'][i]).ravel()
    out_reshaped = out.reshape(channel,int(out.size/channel))
    out_fragment = out_reshaped[:,min_value:min_value+min_period*frequency]
    #change the data to [1,n*18] 
    out_fragment = np.concatenate(out_fragment).ravel()
    # out_fragment = out_fragment.reshape(1,channel*min_period*frequency)
    # eeg_data_list.append(out_fragment)
    data_list.append(out_fragment)
    label_list.append(1)

In [ ]:
#find normal EEG data
min_period = 60
frequency = 200
channel = 18
norm_suit_list = list()#suitable_object_list
for i in range(500):
    signal = f['labels'][i]
    if np.size(np.where(signal==1)) == 0:
        time_period = np.max(np.where(signal==0))-np.min(np.where(signal==0))
        if time_period > min_period*frequency:
            norm_suit_list.append([i,np.min(np.where(signal==0)),np.max(np.where(signal==0))])
# there are 23 objects which is satisfied conditions in first 100 

In [ ]:
#normal data
# normal_data_list = list()#store the data
for tmp_list in norm_suit_list:
    i = tmp_list[0]
    min_value = tmp_list[1]
    max_value = tmp_list[2]
    # solving the special data structure array in array
    out = np.concatenate(f['signals'][i]).ravel()
    out_reshaped = out.reshape(channel,int(out.size/channel))
    out_fragment = out_reshaped[:,min_value:min_value+min_period*frequency]
    out_fragment = np.concatenate(out_fragment).ravel()
    # out_fragment = out_fragment.reshape(1,channel*min_period*frequency)
    # eeg_data_list.append(out_fragment)
    data_list.append(out_fragment)
    label_list.append(0)

In [ ]:
#normalization
def Normalization(x):
    return 10*np.array(x)/np.mean(x)
for i in range(len(data_list)):
    data_list[i] = Normalization(data_list[i])

In [ ]:
#saving the data as .mat file for matlab
import scipy.io as scio
# file_path_1 = 'normal_data_test.mat'
# file_path_2 = 'seizure_data_test.mat'
# scio.savemat(file_path_1, {'Normal_EEG':normal_data_list})
# scio.savemat(file_path_2, {'Seizure_EEG':seizure_data_list})

file_path_1 = 'data_test.mat'
scio.savemat(file_path_1, {'Data':data_list})


In [ ]:
import scipy.io as scio
#loading data
datafile = 'data_test.mat'
# data = scio.loadmat(datafile)
data = scio.loadmat(datafile)['Data']

In [ ]:

#SVM with different variables
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split as ts
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
#import our data

# X = np.nan_to_num(data['final_feature'])
X = data_list
y = label_list

X,y = shuffle(X,y,random_state=0)
# X_1,X_test,y_1,y_test = ts(X,y,test_size=0.3) #分成两份，一份train&eval 一份val
X_train,X_test,y_train,y_test = ts(X,y,test_size=0.3) 
# select different type of kernel function and compare the score
param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]} 

grid_search = GridSearchCV(svm.SVC(), param_grid, cv=2, scoring='accuracy' )    
grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_estimator_.get_params()    
model = svm.SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)    
model.fit(X_train,y_train)  
pred = model.predict(X_test)
score = model.score(X_test,y_test)
print(score)

In [ ]:
#SVM with different variables
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split as ts
from sklearn.utils import shuffle
#import our data

# X = np.nan_to_num(data['final_feature'])
X = data_list
y = label_list

X,y = shuffle(X,y,random_state=0)
# X_1,X_test,y_1,y_test = ts(X,y,test_size=0.3) #分成两份，一份train&eval 一份val
X_train,X_test,y_train,y_test = ts(X,y,test_size=0.6) 
# select different type of kernel function and compare the score


# kernel = 'rbf'
clf_rbf = svm.SVC(kernel='rbf')
clf_rbf.fit(X_train,y_train)
clf_rbf_pred = clf_rbf.predict(X_test)
score_rbf = clf_rbf.score(X_test,y_test)
print("The score of rbf is : %f"%score_rbf)

# kernel = 'linear'
clf_linear = svm.SVC(kernel='linear')
clf_linear.fit(X_train,y_train)
clf_linear_pred = clf_linear.predict(X_test)
score_linear = clf_linear.score(X_test,y_test)
print("The score of linear is : %f"%score_linear)

# kernel = 'poly'
clf_poly = svm.SVC(kernel='poly')
clf_poly.fit(X_train,y_train)
clf_poly_pred = clf_poly.predict(X_test)
score_poly = clf_poly.score(X_test,y_test)
print("The score of poly is : %f"%score_poly)


"""
single column 
The score of rbf is : 0.657143
The score of linear is : 0.628571
The score of poly is : 0.542857
"""

In [ ]:
#SVM & Random Forest & Logistic regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

X,y = shuffle(X,y,random_state=0)
X_train,X_test,y_train,y_test = ts(X,y,test_size=0.3) 

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train,y_train)

for clf in (log_clf,rnd_clf,svm_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

In [ ]:
# KNN clssifier
from sklearn.neighbors import KNeighborsClassifier

X,y = shuffle(X,y,random_state=0)
X_train,X_test,y_train,y_test = ts(X,y,test_size=0.3) 

knn_clf = KNeighborsClassifier(n_neighbors=2)
knn_clf.fit(X_train,y_train)
y_pred = knn_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))


In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
 
# 全局变量
batch_size = 128
nb_classes = 10
epochs = 12
# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)


X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

#构建模型
model = Sequential()
"""
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',
                        input_shape=input_shape))
"""
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='same',
                        input_shape=input_shape)) # 卷积层1
model.add(Activation('relu')) #激活层
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) #卷积层2
model.add(Activation('relu')) #激活层
model.add(MaxPooling2D(pool_size=pool_size)) #池化层
model.add(Dropout(0.25)) #神经元随机失活
model.add(Flatten()) #拉成一维数据
model.add(Dense(128)) #全连接层1
model.add(Activation('relu')) #激活层
model.add(Dropout(0.5)) #随机失活
model.add(Dense(nb_classes)) #全连接层2
model.add(Activation('softmax')) #Softmax评分
 
#编译模型
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
#训练模型
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(X_test, Y_test))
#评估模型
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from scipy import signal

threshold = [2*0.75/200,2*40/200] #2*0.75Hz/200Hz
b, a = signal.butter(4, threshold, 'bandpass')   #配置滤波器 4 表示滤波器的阶数
X = signal.filtfilt(b, a, X)